# Naver Movie 평점 데이터 수집
## 알라딘 Aladdin, 2019

In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

In [2]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=163788'
url = url_base + url_sub

html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

### iframe에서 src 태그의 주소를 얻어야 함

In [3]:
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=163788&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

### review 건수 구하기

In [7]:
url = url_base + ifr_url
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")
count = soup.find('strong', 'total')
count

<strong class="total">관람객 평점 <em>26,438</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [8]:
count = count.find('em').string
count

'26,438'

In [9]:
total_count = int(count.replace(',', ''))
total_count

26438

In [10]:
import math
total_page = math.ceil(total_count / 10)
total_page

2644

### 첫번째 페이지에서 리뷰, 평점, 날짜 구해보기

In [11]:
point = []
review = []
date = []
results = soup.find('div', 'score_result')
lis = results.find_all('li')
for li in lis:
    p = li.find('div', 'star_score').find('em').string
    point.append(p)
    r = li.find('div', 'score_reple').find('p').find('span').string.strip(' \t\n\r')
    review.append(r)
    d = li.find('div', 'score_reple').find_all('em')[1].string
    date.append(d)

### 내용중 관람객 수정 버전

In [39]:
point = []
review = []
date = []
results = soup.find('div', 'score_result')
lis = results.find_all('li')
for li in lis:
    span = li.find('div', 'score_reple').find('p').find_all('span')
#     print(len(span))
    p = li.find('div', 'star_score').find('em').string
    point.append(p)
    if len(span) == 1:
        r = li.find('div', 'score_reple').find('p').find('span').string.strip(' \t\n\r')
    else:
        s = li.find('div', 'score_reple').find('p').find_all('span')[1].string.strip(' \t\n\r')
        if s == None:
            r = li.find('div', 'score_reple').find('p').find_all('span')[1].find('a')['data-src']
        else:
            r = li.find('div', 'score_reple').find('p').find_all('span')[1].string.strip(' \t\n\r')
    review.append(r)
    d = li.find('div', 'score_reple').find_all('em')[1].string
    date.append(d)

- 1. 일반적인 리뷰
- 2. 관람객이라 표시된 경우
- 3. 데이터가 긴 경우
- 4. SPOIL 인 경우
- 5. 관람객이면서 데이터가 긴 경우

In [41]:
point = []
review = []
date = []
for page in range(1, total_page+1):
    url = url_base + ifr_url + '&page=' + str(page)
    html = urlopen(url)
    soup = BeautifulSoup(html, "html.parser")
    if page % 100 == 0:
        print(page, end=' ')
    results = soup.find('div', 'score_result')
    lis = results.find_all('li')
    for li in lis:
        p = li.find('div', 'star_score').find('em').string
        point.append(p)
        s = li.find('div', 'score_reple').find('p').find_all('span')
        if len(s) == 1:         # case 1
            r = li.find('div', 'score_reple').find('span').string.strip(' \t\n\r')
        elif len(s) == 2:
            s = li.find('div', 'score_reple').find('p').find('span').string
            if s == '관람객':   # case 2
                r = li.find('div', 'score_reple').find_all('span')[1].string.strip(' \t\n\r')
            else:
                s = li.find('div', 'score_reple').find('p').find_all('span')[1].string
                if s is None:   # case 3
                    r = li.find('div', 'score_reple').find('p').find_all('span')[1].find('a')['data-src']
                else:           # case 4
                    r = li.find('div', 'score_reple').find('p').find_all('span')[1].string.strip(' \t\n\r')
        else:                   # case 5
            r = li.find('div', 'score_reple').find('p').find_all('span')[2].find('a')['data-src']
        review.append(r)
        d = li.find('div', 'score_reple').find_all('em')[1].string
        date.append(d)    

100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 

In [42]:
import pandas as pd
df = pd.DataFrame({'point':point, 'review':review, 'date':date})
df

,point,review,date
0,10,윌 스미스가 하드캐리하여 의도치 않게 원작을 뛰어넘은 실사영화,2019.05.23 11:26
1,10,영화가 너무 아름답다..ost도 잊혀지지 안고 디즈니 실사중에서 최고중에 최고다 쟈...,2019.05.23 09:11
2,10,존잼임... 난 처음 도입부에 아~라~비안 나~잇 이 부분에서부터 감동받아서 울 뻔...,2019.05.23 21:57
3,10,자스민의 speechless 명장면이고 덤으로 윌 스미스의 연기력 또한 매우 굿~~,2019.05.23 12:55
4,10,A whole new world 나올때때 개소름...,2019.05.23 09:06
5,10,자스민 존나 이쁘다..,2019.05.23 17:36
6,10,지니가 나오면.. 이세상 흥이 아니다ㅎ,2019.05.23 21:54
7,10,일요일 아침 8시 디즈니 만화동산 챙겨보고 비디오 알라딘 애니도 수십번 빌려봐서 테...,2019.05.23 13:06
8,10,디즈니가 미쳤음.. 영화시장을 통재로 잡아먹으려고 작정을함..끝내주는 영화 이제껏 ...,2019.05.23 22:06
9,10,제일 우려스러웠던 윌스미스가 하드캐리하는바람에 원작 뛰어넘어버림 ㅋㅋㅋ,2019.05.24 00:49


In [43]:
df.to_excel('data/04.NaverMovie_Aladdin.xlsx')